## Pendencias:
- Tratar exclusoes lógicas
- Retirar ano dos nomes dos campos (impede que o mesmo codigo seja rodado quando o exercicio vira sem correção manual)

In [1]:
import sys

sys.path.append('C:\\Users\\andre\\OneDrive\\SEPLAG\\2022\\SPLOR\\DCPPN\\projetos\\Identificação de Funções Sociais\\Notebooks\\modules')

from modules.load_data import *
from functools import reduce
from pandas import IndexSlice as idx
import pandas as pd
import numpy as np

    
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:.2f}'.format

## Paths and Filenames

In [2]:
caminho_bancos = 'bases/'
caminho_modulos = 'modules/'
caminho_resultados = 'results/'
caminho_divergencias = ''

nome_arquivo_acoesMon = ''
nome_arquivo_acoesPla = 'acoes_planejamento_2025.txt'
nome_arquivo_locMon = ''
nome_arquivo_exeSiafi = ''
nome_arquivo_invSiafi = ''
nome_arquivo_qddPes = 'qdd item fiscal 2021.xlsx'


nome_funcao_social =  ['Agricultura','Assistência Social','Cultura','Desporto e Lazer','Direitos da Cidadania','Educação','Habitação','Organização Agrária','Previdência Social','Saneamento','Saúde','Segurança Pública','Trabalho','Transporte','Urbanismo','Ciência e tecnologia','Gestão ambiental']
cod_funcao_social =  [20, 8, 13, 27, 14, 12, 16, 21, 9, 17, 10, 6 ,11 ,26 ,15 ,19 ,18 ,23]




## Load Data

In [3]:
acoesPla = load_bases_sigplan(caminho_bancos, nome_arquivo_acoesPla)

init_base_sigplan(acoesPla)

In [4]:
acoesPla

,código_do_programa,nome_do_programa,código_da_área_temática,área_temática,exclusão_lógica_do_programa,programa_novo,justificativa_de_inclusão_ou_exclusão_do_programa,código_da_unidade_orçamentária_responsável_pelo_programa,unidade_orçamentária_responsável_pelo_programa,código_da_unidade_orçamentária_responsável_pela_ação,unidade_orçamentária_responsável_pela_ação,código_da_função,função,código_da_subfunção,subfunção,código_do_tipo_de_ação,tipo_de_ação,código_da_ação,título_da_ação,código_do_identificador_de_ação_governamental_(iag),identificador_de_ação_governamental_(iag),código_do_projeto_estratégico,projeto_estratégico,exclusão_lógica_da_ação,nova_ação,justificativa_de_inclusão_ou_exclusão_da_ação,transferida_para_o_sisor,unidade_administrativa_responsável_pela_ação,base_legal,finalidade_da_ação,descrição_da_ação,código_do_público-alvo,público-alvo,código_do_produto,produto,especificação_do_produto,código_da_unidade_de_medida_do_produto,unidade_de_medida_do_produto,previsão_orçamentária_2025,previsão_orçamentária_2026,previsão_orçamentária_2027,previsão_orçamentária_2028,previsão_física_2025,previsão_física_2026,previsão_física_2027,previsão_física_2028,ação_transposta,setor_de_governo,política_para_mulheres
row_number,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1,MINAS ÍNTEGRA E TRANSPARENTE,13AT13,"Transparência, Combate à Corrupção, Integridad...",Não,Não,NaN,1521,CONTROLADORIA-GERAL DO ESTADO,1521,CONTROLADORIA-GERAL DO ESTADO,4,ADMINISTRAÇÃO,124,CONTROLE INTERNO,4,Atividade Fim do Orçamento Fiscal,4304,PROMOÇÃO DO ACESSO À INFORMAÇÃO E DA TRANSPARÊ...,0,Demais Projetos e Atividades,NaN,NaN,Não,Não,NaN,Não,SUPERINTENDÊNCIA CENTRAL DE TRANSPARÊNCIA,"LEI FEDERAL Nº 12.527 DE 2011, LEI COMPLEMENTA...",PROMOVER AÇÕES PARA CONSOLIDAÇÃO E FORTALECIME...,ENTENDE-SE QUE A FINALIDADE DA AÇÃO SERÁ ATEND...,850,ADMINISTRAÇÃO PÚBLICA ESTADUAL E SOCIEDADE CIVIL,10624,SERVIÇO REALIZADO,SÃO CONSIDERADOS SERVIÇOS DE TRANSPARÊNCIA E A...,222,SERVIÇO,2472869.00,1659917.00,1669127.00,1678583.00,80.00,80.00,80.00,80.00,Não,CONTROLADORIA GERAL,Indiretamente relacionada
1,1,MINAS ÍNTEGRA E TRANSPARENTE,13AT13,"Transparência, Combate à Corrupção, Integridad...",Não,Não,NaN,1521,CONTROLADORIA-GERAL DO ESTADO,1521,CONTROLADORIA-GERAL DO ESTADO,4,ADMINISTRAÇÃO,124,CONTROLE INTERNO,4,Atividade Fim do Orçamento Fiscal,4305,FORTALECIMENTO DA INTEGRIDADE PÚBLICA E PRIVADA,0,Demais Projetos e Atividades,NaN,NaN,Não,Não,NaN,Não,SUPERINTENDÊNCIA CENTRAL DE INTEGRIDADE E CONT...,"DECRETO ESTADUAL Nº 47.674, DE 19 DE JUNHO DE ...",DESENVOLVER AÇÕES PARA CONSOLIDAÇÃO E FORTALEC...,O OBJETO DA AÇÃO CONSISTE NA PROMOÇÃO DA INTEG...,1227,"AGENTES, ÓRGÃOS, ENTIDADES E EMPRESAS PÚBLICAS...",10991,INICIATIVA REALIZADA,SERÃO CONSIDERADAS COMO INICIATIVAS DE PROMOÇÃ...,737,INICIATIVA,15375.00,15784.00,16204.00,16635.00,30.00,30.00,30.00,30.00,Não,CONTROLADORIA GERAL,Indiretamente relacionada
2,1,MINAS ÍNTEGRA E TRANSPARENTE,13AT13,"Transparência, Combate à Corrupção, Integridad...",Não,Não,NaN,1521,CONTROLADORIA-GERAL DO ESTADO,1521,CONTROLADORIA-GERAL DO ESTADO,14,DIREITOS DA CIDADANIA,422,"DIREITOS INDIVIDUAIS, COLETIVOS E DIFUSOS",4,Atividade Fim do Orçamento Fiscal,4306,FOMENTO AO CONTROLE SOCIAL,0,Demais Projetos e Atividades,NaN,NaN,Não,Não,NaN,Não,SUPERINTENDÊNCIA CENTRAL DE INTEGRIDADE E CONT...,CONSTITUIÇÃO DA REPÚBLICA FEDERATIVA DO BRASIL...,ESTIMULAR O APERFEIÇOAMENTO DOS MECANISMOS DE ...,"O OBJETO DA AÇÃO CONSISTE, PARTICULARMENTE, EM...",1227,"AGENTES, ÓRGÃOS, ENTIDADES E EMPRESAS PÚBLICAS...",10991,INICIATIVA REALIZADA,SÃO CONSIDERADAS COMO INICIATIVAS DE FORTALECI...,737,INICIATIVA,10000.00,10266.00,10539.00,10819.00,12.00,12.00,12.00,12.00,Não,CONTROLADORIA GERAL,Indiretamente relacionada
3,2,AUDITORIA INTERNA PARA O FUTURO,13AT13,"Transparência, Combate à Corrupção, Integridad...",Não,Não,NaN,1521,CONTROLADORIA-GERAL DO ESTADO,1521,CONTROLADORIA-GERAL DO ESTADO,4,ADMINISTRAÇÃO,124,CONTROLE INTERNO,4,Atividade Fim do Orçamento Fiscal,430

## Seleciona Colunas Acoes_Planejamento

In [5]:
#retira ações e programas excluídos
df1 = acoesPla.query('exclusão_lógica_do_programa == "Não" and exclusão_lógica_da_ação == "Não"')

#filtra somente colunas necessárias
df1 = df1[['código_do_programa','nome_do_programa', 'código_da_função',  'função', 'previsão_orçamentária_2025' ]]

#define index para merge posterior
df1.set_index(['código_do_programa'], inplace = True)

df1

,nome_do_programa,código_da_função,função,previsão_orçamentária_2025
código_do_programa,,,,
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,2472869.00
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,15375.00
1,MINAS ÍNTEGRA E TRANSPARENTE,14,DIREITOS DA CIDADANIA,10000.00
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,27783.00
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,10000.00
...,...,...,...,...
764,ASSISTÊNCIA COMPLEMENTAR,10,SAÚDE,1000.00
765,APOIO HABITACIONAL,16,HABITAÇÃO,17999000.00
766,BENEFÍCIOS ASSISTENCIAIS,9,PREVIDÊNCIA SOCIAL,4906275.00


## Cria DataFrame com Totais Previsão Orçamentária por Programa

In [6]:
df2 = acoesPla.query('exclusão_lógica_do_programa == "Não" and exclusão_lógica_da_ação == "Não"')

df2 = df2.loc[:,['código_do_programa', 'código_da_função', 'previsão_orçamentária_2025']].groupby(['código_do_programa']).agg(total_prevorc_prog=('previsão_orçamentária_2025', 'sum'))


## Merge df1 e df2 3 cria coluna Categoria da Função (é ou não função social)

In [7]:
df3 = pd.merge(df1, df2, left_on='código_do_programa', right_on='código_do_programa')

#percentual de cada função do total do programa
df3['%'] = (df3['previsão_orçamentária_2025']/df3['total_prevorc_prog'])#.map('{:2.2%}'.format)

#Coluna para indicar se a função é social ou não
df3['categoria_da_função'] = df3['código_da_função'].map(lambda x: 'social' if x in cod_funcao_social else 'não social')

df3

,nome_do_programa,código_da_função,função,previsão_orçamentária_2025,total_prevorc_prog,%,categoria_da_função
código_do_programa,,,,,,,
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,2472869.00,2498244.00,0.99,não social
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,15375.00,2498244.00,0.01,não social
1,MINAS ÍNTEGRA E TRANSPARENTE,14,DIREITOS DA CIDADANIA,10000.00,2498244.00,0.00,social
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,27783.00,164374.00,0.17,não social
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,10000.00,164374.00,0.06,não social
...,...,...,...,...,...,...,...
764,ASSISTÊNCIA COMPLEMENTAR,10,SAÚDE,1000.00,1000.00,1.00,social
765,APOIO HABITACIONAL,16,HABITAÇÃO,17999000.00,17999000.00,1.00,social
766,BENEFÍCIOS ASSISTENCIAIS,9,PREVIDÊNCIA SOCIAL,4906275.00,4906275.00,1.00,social


## Sumariza Linhas por Programa

In [8]:
todos_programas_por_funcao = df3
df4 = df3.groupby(['código_do_programa', 'nome_do_programa', 'categoria_da_função']).agg(Perc_Orçamento = ('%', sum) )

df4

C:\Users\User\AppData\Local\Temp\ipykernel_32724\1692330761.py:2: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df4 = df3.groupby(['código_do_programa', 'nome_do_programa', 'categoria_da_função']).agg(Perc_Orçamento = ('%', sum) )


Perc_Orçamento
código_do_programa nome_do_programa                                   categoria_da_função                
1                  MINAS ÍNTEGRA E TRANSPARENTE                       não social                     1.00
                                                                      social                         0.00
2                  AUDITORIA INTERNA PARA O FUTURO                    não social                     1.00
3                  ENFRENTAMENTO DA CORRUPÇÃO E RECUPERAÇÃO DE ATIVOS não social                     1.00
4                  CGE INTEGRADA E EFICIENTE                          não social                     1.00
5                  PROGRAMA MINEIRO DE ACESSIBILIDADE, INCLUSÃO E ... não social                     0.00
                                                                      social                         1.00
6                  ASSISTÊNCIA À SAÚDE DOS SEGURADOS E DOS SEUS DE... social                         1.00
7                  EDUCAÇÃO SUPERIOR                                  social                         1.00
8                  PRESTAÇÃO DE SERVIÇOS PREVIDENCIÁRIOS              social                         1.00
9                  PROGRAMA DE APOIO HABITACIONAL AOS MILITARES DO... social                         1.00
10                 REGULAÇÃO E FISCALIZAÇÃO DOS SERVIÇOS DE ABASTE... social                         1.00
11                 ASSESSORAMENTO TÉCNICO E ADMINISTRATIVO AO GOVE... não social                     1.00
12                 POLÍTICA PÚBLICA DE METROLOGIA E AVALIAÇÃO DA C... social                         1.00
13                 APORTE DE CAPITAL E MANUTENÇÃO DA INFRAESTRUTUR... não social                     1.00
14                 FINANCIAMENTO À DESCARBONIZAÇÃO E RESILIÊNCIA C... social                         1.00
15                 ENSINO SUPERIOR PARA A AGRICULTURA E PECUÁRIA      social                         1.00
16                 INOVAÇÕES E SOLUÇÕES TECNOLÓGICAS PARA A AGROPE... social                         1.00
17                 INVESTIMENTO NA INFRAESTRUTURA ADMINISTRATIVA E... social                         1.00
18                 COMUNICAÇÃO INSTITUCIONAL E DE UTILIDADE PÚBLICA   não social                     1.00
19                 ATENÇÃO HOSPITALAR ESPECIALIZADA                   social                         1.00
20                 FOMENTO E INCENTIVO DE INVESTIMENTOS               não social                     1.00
21                 SANEAMENTO E RESÍDUOS ESPECIAIS                    social                         1.00
22                 CONSTRUINDO O FUTURO POR MEIO DA CIÊNCIA, DA TE... social                         1.00
23                 POLÍTICAS PARA CRIANÇAS E ADOLESCENTES             social                         1.00
24                 DESENVOLVIMENTO SOCIOECONÔMICO                     social                         1.00
25                 MONITORAMENTO, CONTROLE E FISCALIZAÇÃO AMBIENTAL   social                         1.00
26                 DESENVOLVIMENTO DO ENSINO SUPERIOR NA UEMG         social                         1.00
27                 GESTÃO AMBIENTAL.                                  social                         1.00
28                 DESENVOLVIMENTO DE AÇÕES DE EDUCAÇÃO E PESQUISA... social                         1.00
29                 REGISTRO PÚBLICO DE EMPRESAS MERCANTIS E ATIVID... social                         1.00
30                 SOLUÇÕES EM TECNOLOGIA DA INFORMAÇÃO E COMUNICAÇÃO não social                     1.00
31                 PROTEÇÃO DAS ÁREAS AMBIENTALMENTE CONSERVADAS, ... social                         1.00
32                 INVESTIGAÇÃO                                       social                         1.00
33                 GESTÃO DA INFORMAÇÃO COM USO DE TECNOLOGIA E CO... não social                     1.00
34                 SUPORTE ÀS AÇÕES DE TRANSPORTE RODOVIÁRIO          social                         1.00
35                 INVESTIMENTOS CEMIG                         

## Filtra Programas Sociais

In [9]:
programas_sociais = df4.query('categoria_da_função == "social" and Perc_Orçamento > 0.5')
#programas_sociais.reset_index(inplace=True)

#Salva Programais Sociais em um arquivo csv na pasta /results
programas_sociais.to_csv(caminho_resultados + 'programas_sociais.csv', sep=';', decimal=',', encoding='latin-1')
programas_sociais

,,,Perc_Orçamento
código_do_programa,nome_do_programa,categoria_da_função,
5,"PROGRAMA MINEIRO DE ACESSIBILIDADE, INCLUSÃO E SAÚDE (PROMAIS)",social,1.00
6,ASSISTÊNCIA À SAÚDE DOS SEGURADOS E DOS SEUS DEPENDENTES,social,1.00
7,EDUCAÇÃO SUPERIOR,social,1.00
8,PRESTAÇÃO DE SERVIÇOS PREVIDENCIÁRIOS,social,1.00
9,PROGRAMA DE APOIO HABITACIONAL AOS MILITARES DO ESTADO DE MINAS GERAIS,social,1.00
10,REGULAÇÃO E FISCALIZAÇÃO DOS SERVIÇOS DE ABASTECIMENTO DE ÁGUA E DE ESGOTAMENTO SANITÁRIO E ATENDIMENTO AOS USUÁRIOS,social,1.00
12,POLÍTICA PÚBLICA DE METROLOGIA E AVALIAÇÃO DA CONFORMIDADE,social,1.00
14,FINANCIAMENTO À DESCARBONIZAÇÃO E RESILIÊNCIA CLIMÁTICA ENTRE BDMG E BANCO INTERAMERICANO DE DESENVOLVIMENTO (BDMG PBR),social,1.00
15,ENSINO SUPERIOR PARA A AGRICULTURA E PECUÁRIA,social,1.00


In [10]:
programas_sociais.count()

Perc_Orçamento    120
dtype: int64

In [11]:
todos_programas_por_funcao.to_csv(caminho_resultados + 'CONFERENCIA - todos_programas_por_funcao.csv', sep=';', decimal=',', encoding='latin-1')
todos_programas_por_funcao

,nome_do_programa,código_da_função,função,previsão_orçamentária_2025,total_prevorc_prog,%,categoria_da_função
código_do_programa,,,,,,,
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,2472869.00,2498244.00,0.99,não social
1,MINAS ÍNTEGRA E TRANSPARENTE,4,ADMINISTRAÇÃO,15375.00,2498244.00,0.01,não social
1,MINAS ÍNTEGRA E TRANSPARENTE,14,DIREITOS DA CIDADANIA,10000.00,2498244.00,0.00,social
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,27783.00,164374.00,0.17,não social
2,AUDITORIA INTERNA PARA O FUTURO,4,ADMINISTRAÇÃO,10000.00,164374.00,0.06,não social
...,...,...,...,...,...,...,...
764,ASSISTÊNCIA COMPLEMENTAR,10,SAÚDE,1000.00,1000.00,1.00,social
765,APOIO HABITACIONAL,16,HABITAÇÃO,17999000.00,17999000.00,1.00,social
766,BENEFÍCIOS ASSISTENCIAIS,9,PREVIDÊNCIA SOCIAL,4906275.00,4906275.00,1.00,social
